# Actualizacion de Tablas SQL
## Utilizando las nuevas tablas desde el modulo de sistematizacion

Antes que todo, es necesario que la carpeta `db/CSV/` se encuentra con los archivos nuevos -i.e., copiar y reemplazar en la carpeta las tablas nuevas.

En particular, las nuevas tablas contienen procesada la columna `macro`. Esta columna asigna una etiqueta o categoria, permitiendo agrupar distintas respuestas. 

In [39]:
import pandas as pd
import numpy as np
import os 


from use_cases.pairs import create_pair_token
from use_cases.utils.textools import clean_alt_list

from use_cases.contributions import create_table_contributions, to_sql as con_to_sql
from use_cases.emotions import create_table_emotions, to_sql as emo_to_sql 
from use_cases.personal_needs import create_table_personal_needs, to_sql as per_need_to_sql
from use_cases.country_needs import create_table_country_needs, to_sql as count_need_to_sql

### Emociones y Sentimientos

In [31]:
emotions_table = pd.read_csv('./db/CSV/emotions.csv', low_memory=False)

In [32]:
emotions_table.sample(1)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online
12616,12616,ENC_U_4545075889151833611,NaN,impotencia,['impotencia'],Tristeza,"destruyo todo y de nosotros sale la plata, hoy...","['destruyo', 'sale', 'plata', 'hoy', 'ley', 'm...",True


las listas se guardan como `strings` en el csv, es necesario volver a dejarlos como `listas`

In [42]:
emotions_table['exp_tokens'] = emotions_table['exp_tokens'].apply(clean_alt_list)
emotions_table = emotions_table.replace(np.nan, '')

Debemos recalcular los pares de palabras en caso de que algo haya cambiado

In [34]:
emotion_pair = create_pair_token(emotions_table, 'exp_tokens', 'emotion_id')

guardamos/reemplazamos la nueva tabla de pares 

In [35]:
emotion_pair.to_csv('./db/CSV/emotions_pairs.csv', index=False)

y finalmente los nuevos `.sql`

In [49]:
emotion_pair.to_csv('./db/CSV/emotions_pairs.csv.csv', index=False)

In [44]:
emo_to_sql(emotions_table, './db/SQL/emotions.sql')

### Contribuciones

En el caso de contribuciones solo necesitamos guardar el nuevo `csv` como `sql`

In [46]:
contributions_table = pd.read_csv('./db/CSV/contributions.csv', low_memory=False)

In [47]:
contributions_table.sample(1)

,Unnamed: 0,id,diag_id,ind_id,text,tokens,macro,is_online
43966,43966,50059,ENC_U_4537991899159681147,NaN,"sido honesto , la educacion , que el sistema b...","['sido', 'honesto', 'educacion', 'sistema', 'b...",Reciprocidad-Redes,False


In [50]:
contributions_table['tokens'] = contributions_table['tokens'].apply(clean_alt_list)
contributions_table = contributions_table.replace(np.nan, '')

In [52]:
con_to_sql(contributions_table, './db/SQL/contribution.sql')